In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split 

In [3]:
# --- 1. Định nghĩa đường dẫn ---
DATA_DIR = r"E:\HANDS-VNOnDB" # <--- Đảm bảo đường 

--- 1. Thiết lập cấu hình ---


In [4]:
# --- Siêu tham số Mô hình & Huấn luyện ---
TARGET_IMG_HEIGHT = 32    # Phải khớp với ảnh đã tiền xử lý
TARGET_IMG_WIDTH = 256    # Phải khớp với ảnh đã tiền xử lý
CNN_OUTPUT_WIDTH = TARGET_IMG_WIDTH // 4 # Ước lượng SeqLen từ CNN
RNN_HIDDEN_SIZE = 256     # Kích thước ẩn LSTM
RNN_NUM_LAYERS = 1        # Số lớp LSTM (giảm để nhanh hơn)
LEARNING_RATE = 5e-5      # Learning rate thấp
WEIGHT_DECAY = 0          # Không dùng weight decay ban đầu
BATCH_SIZE = 16           # Batch size (điều chỉnh nếu cần)
NUM_EPOCHS = 5          # Số epoch tối đa
NUM_WORKERS = 0           # Tốt nhất cho CPU/Windows
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# --- Siêu tham số Early Stopping ---
PATIENCE = 7              # Số epoch chờ cải thiện Val CER (tăng lên chút)
MIN_DELTA = 0.005         # Mức cải thiện CER tối thiểu

print(f"   Sử dụng Device: {DEVICE}")
print(f"   Kích thước ảnh: {TARGET_IMG_WIDTH}x{TARGET_IMG_HEIGHT}")
print(f"   Batch Size: {BATCH_SIZE}")
print(f"   Learning Rate: {LEARNING_RATE}")
print(f"   Epochs tối đa: {NUM_EPOCHS}")
print(f"   Early Stopping Patience: {PATIENCE}")

   Sử dụng Device: cpu
   Kích thước ảnh: 256x32
   Batch Size: 16
   Learning Rate: 5e-05
   Epochs tối đa: 5
   Early Stopping Patience: 7


In [5]:
# --- 2. Load Bản đồ ký tự và Dữ liệu CSV ---
print("\n--- 2. Tải Bản đồ ký tự và Dữ liệu CSV ---")
try:
    # Load char map
    with open(CHARMAP_PATH, 'r', encoding='utf-8') as f_map:
        char_maps = json.load(f_map)
        char_to_int = char_maps['char_to_int']
        int_to_char = {int(k): v for k, v in char_maps['int_to_char'].items()} # Key là int
    num_classes_htr = len(char_to_int)
    BLANK_TOKEN = '<blank>'
    BLANK_TOKEN_INDEX = char_to_int.get(BLANK_TOKEN, 0)
    print(f"   Đã tải bản đồ ký tự. Số lớp: {num_classes_htr}, Blank index: {BLANK_TOKEN_INDEX}")

    # Load CSV files
    train_df = pd.read_csv(TRAIN_CSV_PATH)
    val_df = pd.read_csv(VAL_CSV_PATH)
    test_df = pd.read_csv(TEST_CSV_PATH)
    # Lấy list paths và text labels
    X_train_paths = train_df['image_path'].tolist()
    y_train_text = train_df['label'].fillna('').tolist() # Xử lý NaN nếu có
    X_val_paths = val_df['image_path'].tolist()
    y_val_text = val_df['label'].fillna('').tolist()
    X_test_paths = test_df['image_path'].tolist()
    y_test_text = test_df['label'].fillna('').tolist()
    print(f"   Đã tải dữ liệu từ CSV: Train={len(X_train_paths)}, Val={len(X_val_paths)}, Test={len(X_test_paths)}")

except FileNotFoundError as e:
    print(f"Lỗi: Không tìm thấy file cần thiết: {e}.")
    exit()
except Exception as e:
    print(f"Lỗi không xác định khi tải dữ liệu: {e}")
    exit()


--- 2. Tải Bản đồ ký tự và Dữ liệu CSV ---
   Đã tải bản đồ ký tự. Số lớp: 162, Blank index: 0
Lỗi: Không tìm thấy file cần thiết: [Errno 2] No such file or directory: 'E:\\HANDS-VNOnDB\\validation_htr_lines.csv'.
